In [0]:
import pandas as pd
import numpy as np
import warnings
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from imblearn.pipeline import Pipeline, make_pipeline
from imblearn.over_sampling import SMOTE


trainCSV = "https://raw.githubusercontent.com/andybbruno/DataMining/master/new_train_cleaned.csv?token=AGWKQXZRZZDVJOUEMOQ3PJC6AHO6U"

In [13]:
# READ AND DROP COLUMNS
df = pd.read_csv(trainCSV)
y = df['IsBadBuy']
X = df.drop(columns=['IsBadBuy'])

# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

# Set the parameters by cross-validation
params = {
    'criterion' : ['gini', 'entropy'],
    'max_depth': [2, 5, 10, 15, None],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 5, 10, 20],
}

new_params = {'decisiontreeclassifier__' + key: params[key] for key in params}

#scores = ['precision', 'recall', 'f1']
scores = ['recall', 'f1']

for score in scores:
    print("# Tuning hyper-parameters for ----> %s" % score)
    print()
    
    obj = make_pipeline(SMOTE(), tree.DecisionTreeClassifier())
    
    if (score == "recall"):
      clf = GridSearchCV(obj, param_grid=new_params, cv=5, scoring=score)
    else:
      clf = GridSearchCV(obj, param_grid=new_params, cv=5, scoring='%s_macro' % score)
                         
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

# Tuning hyper-parameters for ----> recall

Best parameters set found on development set:

{'decisiontreeclassifier__criterion': 'gini', 'decisiontreeclassifier__max_depth': 2, 'decisiontreeclassifier__min_samples_leaf': 10, 'decisiontreeclassifier__min_samples_split': 2}

Grid scores on development set:

0.621 (+/-0.367) for {'decisiontreeclassifier__criterion': 'gini', 'decisiontreeclassifier__max_depth': 2, 'decisiontreeclassifier__min_samples_leaf': 1, 'decisiontreeclassifier__min_samples_split': 2}
0.622 (+/-0.400) for {'decisiontreeclassifier__criterion': 'gini', 'decisiontreeclassifier__max_depth': 2, 'decisiontreeclassifier__min_samples_leaf': 1, 'decisiontreeclassifier__min_samples_split': 5}
0.625 (+/-0.386) for {'decisiontreeclassifier__criterion': 'gini', 'decisiontreeclassifier__max_depth': 2, 'decisiontreeclassifier__min_samples_leaf': 1, 'decisiontreeclassifier__min_samples_split': 10}
0.533 (+/-0.466) for {'decisiontreeclassifier__criterion': 'gini', 'decisiontreeclassi